# Problem 1
Use Naive Gaussian Elimination to solve 
\begin{aligned} 2 x+y-4 z &=-7 \\ x-y+z &=-2 \\-x+3 y-2 z &=6 \end{aligned}

In [1]:
import numpy as np

def back_sub(A, b):
    """
    Backward substitution
    The right hand side b is changed in place to store the solution
    A: the coefficient matrix of size n x n
    b: the right hand side of size n
    """
    if A.shape[0] != A.shape[1]:
        print('Error: the given coefficient matrix is not square')
        return
    
    if A.shape[0] != b.size:
        print('Error: the shape of the coefficient matrix does not match the size of the RHS')
        return
     
    n = A.shape[0]
    
    for i in range(n-1, -1, -1):
        for j in range(i+1, n):
            b[i] = b[i] - A[i,j]*b[j]
        b[i] = b[i]/A[i,i]

def GaussianElim(A, b):
    """
    Gaussian Elimination without pivoting. 
    The coefficient matrix A and the right hand side b are modified in place.
    Input:
    A: the coefficient matrix of size n x n
    b: the right hand side of size n    
    Return:
    None. The solution is stored in b
    """
    if A.shape[0] != A.shape[1]:
        print('Error: the given coefficient matrix is not square')
        return
    
    if A.shape[0] != b.size:
        print('Error: the shape of the coefficient matrix does not match the size of the RHS')
        return
     
    # We hard code the epsilon here. It can be an input parameter.
    eps = 1e-5 
    n = A.shape[0]

    for j in range(n-1):
        if np.abs(A[j,j]) < eps:
            print('Error: zero pivot encountered!')
            return
        for i in range(j+1, n):
            # The multiplier
            mp = A[i,j]/A[j,j]
            for k in range(j+1,n):
                A[i,k] = A[i,k] - mp*A[j,k]
            b[i] = b[i] - mp*b[j]
    
    # No need to return. Both A and b are changed in place
    

# OUTPUT 
A = np.array([[2., 1, -4], [1, -1, 1], [-1, 3, -2]])
b = np.array([-7, -2, 6])
print('Solve the linear system: ')
print(A, 'x = ', b)
GaussianElim(A, b)
back_sub(A, b)
print('The solution is ', b)

Solve the linear system: 
[[ 2.  1. -4.]
 [ 1. -1.  1.]
 [-1.  3. -2.]] x =  [-7 -2  6]
The solution is  [-2  1  1]


# Problem 2

Use the $LU$ factorization to solve
$$
\left[\begin{array}{lll}{4} & {2} & {0} \\ {4} & {4} & {2} \\ {2} & {2} & {3}\end{array}\right]\left[\begin{array}{l}{x_{1}} \\ {x_{2}} \\ {x_{3}}\end{array}\right]=\left[\begin{array}{l}{2} \\ {4} \\ {6}\end{array}\right]
$$

In [3]:
def LU(A):
    """
    LU factorization without pivoting. 
    The coefficient matrix A is modified in place.
    The lower triangular part of A represents the L matrix, the upper triangular part 
    (including the diagonal) represents U
    A: the coefficient matrix of size n x n   
    """
    if A.shape[0] != A.shape[1]:
        print('Error: the given coefficient matrix is not square')
        return   
     
    # We hard code the epsilon here. It can be an input parameter.
    eps = 1e-5 
    n = A.shape[0]

    for j in range(n-1):
        if np.abs(A[j,j]) < eps:
            print('Error: zero pivot encountered!')
            return
        for i in range(j+1, n):
            # The multiplier
            mp = A[i,j]/A[j,j]
            A[i,j] = mp
            for k in range(j+1,n):
                A[i,k] = A[i,k] - mp*A[j,k]
    
    # No need to return. A is changed in place.

def forward_sub(A, b, A_from_LU):
    """
    Forward substitution
    The right hand side b is changed in place to store the solution
    A: the coefficient matrix of size n x n
    b: the right hand side of size n
    A_from_LU: True, if the matrix A is from LU factorization (diagonals are 1).
               False, if A is just a regular coefficient matrix
    """
    if A.shape[0] != A.shape[1]:
        print('Error: the given coefficient matrix is not square')
        return
    
    if A.shape[0] != b.size:
        print('Error: the shape of the coefficient matrix does not match the size of the RHS')
        return
     
    n = A.shape[0]
    
    if A_from_LU:
        for j in range(0,n):       
            b[j+1:] = b[j+1:] - A[j+1:,j]*b[j]
    else:
        for j in range(0,n):  
            b[j] = b[j]/A[j,j]
            b[j+1:] = b[j+1:] - A[j+1:,j]*b[j]

# OUTPUT 
A = np.array([[4.,2,0], [4,4,2], [2, 2, 3]])
b = np.array([2,4,6])
# Perform LU factorization
LU(A)
print('The matrix A after LU decomposition is ')
print(A)
# Perform forward substitution
forward_sub(A,b,True)
print('After forward substition, the vector b is')
print(b)
# Perform back substitution
back_sub(A,b)
print('After backward substition, the vector b is')
print(b)

The matrix A after LU decomposition is 
[[4.  2.  0. ]
 [1.  2.  2. ]
 [0.5 0.5 2. ]]
After forward substition, the vector b is
[2 2 4]
After backward substition, the vector b is
[ 1 -1  2]


# Problem 3
Let $A$ be the $n\times n$ matrix with entries $A_{ij}=|i-j|+1$. Define $x=[1,\dots,1]^T$ and $b=Ax$. For $n=100,200,300,400$ and $500$, use the Python function numpy.linalg.solve to compute $x_c$, the double precision computed solution. For each solution, calculate the infinity norm of the forward error, find the error magnification factor, and compare with the corresponding condition numbers.

In [11]:
#Condition Number func
def cond(n):
    H = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            H[i,j] = 1./((i+1)+(j+1)-1.)
    b = np.dot(H,np.ones(n,))

    print('The condition number of H is: {0:12.9e}'.format(np.linalg.cond(H)))

    x = np.linalg.solve(H, b)
    print('The solution x is: ')
    print("[{0:19.16f} {1:19.16f} {2:19.16f} {3:19.16f} "
          "{4:19.16f} {4:19.16f} {4:19.16f} {4:19.16f}]".format(x[0], x[1], x[2], 
          x[3], x[4], x[5], x[6], x[7]))
    print('The error vector of the solution is: ')
    print("[{0:19.16e} {1:19.16e} {2:19.16e} {3:19.16e} {4:19.16e} {4:19.16e} "
          "{4:19.16e} {4:19.16e}]".format(np.abs(x[0]-1), np.abs(x[1]-1), np.abs(x[2]-1), 
                                          np.abs(x[3]-1), np.abs(x[4]-1), np.abs(x[5]-1),
                                          np.abs(x[6]-1), np.abs(x[7]-1)))
print("FOR N=100")
print("---------")
cond(100)
print("FOR N=200")
print("---------")
cond(200)
print("FOR N=300")
print("---------")
cond(300)
print("FOR N=400")
print("---------")
cond(400)
print("FOR N=500")
print("---------")
cond(500)

FOR N=100
---------
The condition number of H is: 1.075338660e+19
The solution x is: 
[ 1.0000002066083167  0.9999731101662166  1.0006354937840984  1.0001428923954447  0.8578496473209455  0.8578496473209455  0.8578496473209455  0.8578496473209455]
The error vector of the solution is: 
[2.0660831667740354e-07 2.6889833783405592e-05 6.3549378409843982e-04 1.4289239544473809e-04 1.4215035267905451e-01 1.4215035267905451e-01 1.4215035267905451e-01 1.4215035267905451e-01]
FOR N=200
---------
The condition number of H is: 2.995752372e+20
The solution x is: 
[ 1.0000006391373608  0.9998628138457037  1.0062411631902834  0.8930648795336527  1.8072457288311459  1.8072457288311459  1.8072457288311459  1.8072457288311459]
The error vector of the solution is: 
[6.3913736081389061e-07 1.3718615429625380e-04 6.2411631902834497e-03 1.0693512046634734e-01 8.0724572883114587e-01 8.0724572883114587e-01 8.0724572883114587e-01 8.0724572883114587e-01]
FOR N=300
---------
The condition number of H is: 1.0816

In [12]:
# The condition number increases as the value n increases. 

# Problem 4
Solve the following system by PA=LU factorization. 
$$
\begin{bmatrix}
-9 & 1 & 17 \\
3 & 2 & -1 \\
6 & 8 & 1
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
5 \\ 9 \\-3
\end{bmatrix}
$$

In [4]:
def PALU(A):
    """
    PA=LU factorization with partial pivoting. 
    The coefficient matrix A is modified in place.
    The lower triangular part of A represents the L matrix, the upper triangular part 
    (including the diagonal) represents U
    A: the coefficient matrix of size n x n
    output:
    Permutation matrix $P$ is returned, along with $A$ that is changed in place
    """
    if A.shape[0] != A.shape[1]:
        print('Error: the given coefficient matrix is not square')
        return   
     
    n = A.shape[0]
    P = np.eye(n)

    for j in range(n-1):
        # find p
        p = np.argmax(np.abs(A[j:,j]))        
        if p+j != j:
            # change rows p and j of A. Update P:
            A[[p+j, j]] = A[[j, p+j]]
            P[[p+j, j]] = P[[j, p+j]]   
        for i in range(j+1, n):
            # The multiplier
            mp = A[i,j]/A[j,j]
            A[i,j] = mp
            for k in range(j+1,n):
                A[i,k] = A[i,k] - mp*A[j,k]
    return P

# OUTPUT
A = np.array([[-9., 1, 17], [3, 2, -1], [6, 8, 1]])
b = np.array([5., 9, -3])
P = PALU(A)
# First solve for c
Pb = np.dot(P,b)
forward_sub(A, Pb, True)
# Solve for x
back_sub(A, Pb)
print('The solution is: ')
print(Pb)

The solution is: 
[ 13.04761905 -11.14285714   7.85714286]


# Problem 5 (MATH 5660 ONLY)
**Gaussian Elimination with Scaled Row Pivoting**

We now describe an algorithm called *Gaussian elimination with scaled row pivoting* that also leads to a $PA=LU$ factorization. 

We begin by computing the scale of each row. We put
$$
s_i = \text{max}_{1\le j \le n}|a_{ij}| = \text{max}\{|a_{i1}|, |a_{i2}|, \dots, |a_{in}|\}\,\, (1\le i\le n)
$$
These are recorded in an array $s$ in the algorithm.

In starting the factorization phase, we do not arbitrarily subtract multiples of row one from the other rows. Rather, we select as the pivot row the row for which $|a_{i1}|/s_i$ is largest. The index thus chosen is denoted by $p_1$ and becomes the first entry in the permutation array. Thus, $|a_{p_11}|/s_{p_1}\ge |a_{i1}|/s_i$, for $1\le i\le n$. Once $p_1$ has been determined, we subtract appropriate multiples of row $p_1$ from the other rows in order to produce zeros in the first column of $A$. Of course, row $p_1$ will remain unchanged throughout the remainder of the factorization process.

To keep track of the indices $p_i$ that arise, we begin by setting the permutation vector $(p_1,p_2,\dots,p_n)$ to $(1,2,\dots,n)$. Then we select an index $j$ for which $|a_{p_j1}|/s_{p_j}$ is maximal, and interchange $p_1$ with $p_j$ in the permutation array $p$. The actual elimination step involves subtracting $(a_{p_i1/a_{p_11}})$ times row $p_1$ from row $p_i$ for $2\le i\le n$.

To describe the general process, suppose that we are ready to create zeros in column $k$. We scan the numbers $|a_{p_ik}|/s_{p_i}$ for $k\le i\le n$ looking for a maximal entry. If $j$ is the index of the first of the largest of these ratios, we interchange $p_k$ with $p_j$ in the array $p$, and then subtract $(a_{p_ik}/a_{p_kk})$ times row $p_k$ from row $p_i$ for $k+1\le i\le n$.

Here is how this works on the matrix
$$
A = 
\begin{bmatrix}
2 & 3 & -6 \\
1 & -6 & 8 \\
3 & -2 & 1
\end{bmatrix}
$$
At the beginning, $p=(1,2,3)$ and $s=(6,8,3)$. To select the first pivot row, we look at the ratios $\{2/6, 1/8, 3/3\}$. The largest corresponds to $j=3$, and row $3$ is to be the first pivot row. So we interchange $p_1$ with $p_3$, obtaining $p=(3,2,1)$. Now multiples of row $3$ are subtracted from rows  $2$ and $1$ to produce zeros in the first column. The result is
$$
\begin{bmatrix}
\frac{2}{3} & \frac{13}{3} & -\frac{20}{3} \\
\frac{1}{3} & -\frac{16}{3} & \frac{23}{3} \\
3 & -2 & 1
\end{bmatrix}
$$
The entries in locations $a_{11}$ and $a_{21}$ are the multipliers. In the next step, the selection of a pivot row is made on the basis of the numbers $|a_{p_22}|/s_{p_2}$ and $|a_{p_32}|/s_{p_3}$. The first of these ratios is $(16/3)/8$ and the second is $(13/3)/6$. So $j=3$, and we interchange $p_2$ with $p_3$. Then a multiple of row $p_2$ is subtracted from row $p_3$. The result is $p=(3,1,2)$ and
$$
\begin{bmatrix}
\frac{2}{3} & \frac{13}{3} & -\frac{20}{3} \\
\frac{1}{3} & -\frac{16}{13} & -\frac{7}{13} \\
3 & -2 & 1
\end{bmatrix}
$$
The final multiplier is stored in location $a_{22}$.

If the rows of the original matrix $A$ were interchanged according to the final permutation array $p$, then we would have an LU factorization of $A$. Hence, we have 
$$
\begin{bmatrix}
3 & -2 & 1 \\
2 & 3 & -6 \\
1 & -6 & 8 \\
\end{bmatrix} = PA =
\begin{bmatrix}
1 & 0 & 0 \\
\frac{2}{3} & 1 & 0 \\
\frac{1}{3} & -\frac{16}{3} & 1
\end{bmatrix}
\begin{bmatrix}
3 & -2 & 1 \\
0 & \frac{13}{3} & -\frac{20}{3} \\
0 & 0 & -\frac{7}{13} \\
\end{bmatrix} = LU
$$

Write a function GESRP(A), with $A$ changed in place and $p$ returned, to perform Gaussian Elimination with Scaled Row Pivoting. Test your code with the example given above.